## 1.4 Python, ScikitLearn, PyTorch

### 1.4.1 Pythonについて
- 1991 年初出
    - 言語としては古参
    - 継続的に進化し続けているが、言語仕様、実装ともにやや古典的
- いくつか実装があるが、典型的にはCPythonを用いる。
    - いわゆるインタプリタ言語だが、厳密にはpycというバイトコード形式にコンパイルされる
- 一般に**非常に遅い**
    - C言語と比べると50-100倍ぐらい
    - マルチスレッドがまともに動かない
        - Giant Interpreter Lock
        - 事実上1スレッドしか同時には実行できない
- にもかかわらず広く用いられている - なぜか
    - 多用されるライブラリの多くはC/C++でコア部分が書かれている
    - Pythonはラッパにすぎない

### 1.4.2 Pythonの速度
- 実際どのくらい遅いのだろうか
- 簡単なプログラムで測ってみる
    - 10000000 (1千万)の1を単純なループですべて足し合わせる。

In [2]:
# Pythonのコード

import time
l = [1] * 10000000;
before = time.time()
acc = 0
for i in l:
    acc += i
time.time() - before

0.7713139057159424

In [3]:
# Cのコード
!cat test.c

#include <stdio.h>
#include <time.h>
#define N 10000000

int buf[N];

int main() {
    struct timespec ts_before, ts_after;
    for (int i = 0; i < N; i++) buf[i] = 1;

    clock_gettime(CLOCK_REALTIME, &ts_before);
    int acc = 0;
    for (int i = 0; i < N; i++) 
        acc += buf[i];
    clock_gettime(CLOCK_REALTIME, &ts_after);
    double diff = (ts_after.tv_sec - ts_before.tv_sec) +
                ((ts_after.tv_nsec - ts_before.tv_nsec) / 1000000000.0);
    printf("%d, %lf\n", acc, diff);
}

In [4]:
!gcc -O3 test.c
!./a.out

10000000, 0.002381


- ざっくりと50-100倍ぐらい遅いことがわかる
- ではなぜPythonを使うのか
    - ライブラリを使うのが前提だから
    - 同じことをnumpyでやってみる

In [6]:
import numpy as np

a = np.ones(10000000)
before = time.time()
a.sum()
time.time() - before

0.007796049118041992

### 1.4.3 Numpyはなぜ速いのか
- numpy はデータをPythonのデータ構造としてではなく、独自の格納方法で、自分で管理している領域に保持する
- 計算(ここでは `sum` )もPythonのルーチンではなく、C言語で書かれた独自の実装を用いる
- 要するにPythonはインターフェイスを提供しているだけ
- 他のライブラリもすべて同じ。基本的にラッパ

### Python Tips
- ライブラリの提供するデータ構造を使う
    - numpy のArray
    - pandas の Dataframe
    - PytorchのTensor
- ループを書かない
    - Pythonのループを書いたら負け
    

### 1.4.4 本稿で使用するライブラリ
- numpy 数値演算ライブラリ
- scipy 科学技術計算ライブラリ
- pandas テーブルデータをDataFrameと呼ばれるデータ構造で管理
- matplotlib グラフのプロットに用いる
- graphviz ノードとエッジで構成されれたグラフ構造を描画
- scikit-learn 機械学習ライブラリの集合体
- pytorch ニューラルネットワークライブラリ
- fastai pytorch を更に使いやすくするためのサポートライブラリ



### Scikit-learn
- 機械学習に関するライブラリの集合体
    - https://scikit-learn.org/
- さまざまなアルゴリズムに対して統一的なインターフェイスを提供
    - fit - 訓練
    - predict - 予測
    - transform - 変換
- さまざまなアルゴリズムを切り替えてテストすることが容易

### PyTorch
- ニューラルネットワークライブラリ
    - 一時は林立したNNライブラリだが、現在はこのPyTorchとGoogle の TensorFlowに収斂
    - 2016年初出と後発 (TensorFlow は2015年)
    - もともとはTorchというライブラリ(2002年初出)で、Lua言語向けに記述されていたが、Python 向けに向けに構成し直したことで高速に普及
- Facebook(Meta) がスポンサー

